<a href="https://colab.research.google.com/github/Jkt1412/BiRnn-Attention-Summarizer/blob/master/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get BBC Dataset

In [2]:
from google.colab import drive
drive.mount("./gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [3]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Text\ Summarization

/content/gdrive/My Drive/Colab Notebooks/Datasets/Text Summarization


In [0]:
import pickle 
with open('myfile.pkl','rb') as fp:
  heading, article = pickle.load(fp)

In [0]:
article[1]

'New Product Gives Marketers Access to Real Keywords, Conversions and Results Along With 13 Months of Historical Data \n\nSAN FRANCISCO, CA -- (Marketwired) -- 09/17/15 -- Jumpshot, a marketing analytics company that uses distinctive data sources to paint a complete picture of the online customer journey, today announced the launch of Jumpshot Elite, giving marketers insight into what their customers are doing the 99% of the time they\'re not on your site. For years, marketers have been unable to see what organic and paid search terms users were entering, much less tie those searches to purchases. Jumpshot not only injects that user search visibility back into the market, but also makes it possible to tie those keywords to conversions -- for any web site. \n\n"Ever since search engines encrypted search results, marketers have been in the dark about keywords, impacting not only the insight into their own search investments, but also their ability to unearth high converting keywords for 

In [0]:
heading[1]

'Jumpshot Gives Marketers Renewed Visibility Into Paid and Organic Keywords With Launch of Jumpshot Elite'

**Data from BBC News Dataset Loaded. The variables are *heading* and *article***

# Preprocess the Data , remove the \n\n and \xa0 , lower case everything  (stemming/lemmatization advantageous? Try it out later)

## I think I should keep the comma's in for now , maybe the Attention mechanism later would find words near a comma a good place to give attention to

In [0]:
heading = [h.lower() for h in heading]
article = [a.lower() for a in article]

In [0]:
article = [a.replace('\n','').replace('\xa0',' ') for a in article]

In [0]:
article[1]

'new product gives marketers access to real keywords, conversions and results along with 13 months of historical data san francisco, ca -- (marketwired) -- 09/17/15 -- jumpshot, a marketing analytics company that uses distinctive data sources to paint a complete picture of the online customer journey, today announced the launch of jumpshot elite, giving marketers insight into what their customers are doing the 99% of the time they\'re not on your site. for years, marketers have been unable to see what organic and paid search terms users were entering, much less tie those searches to purchases. jumpshot not only injects that user search visibility back into the market, but also makes it possible to tie those keywords to conversions -- for any web site. "ever since search engines encrypted search results, marketers have been in the dark about keywords, impacting not only the insight into their own search investments, but also their ability to unearth high converting keywords for their co

In [0]:
heading[1]

'jumpshot gives marketers renewed visibility into paid and organic keywords with launch of jumpshot elite'

# Toeknize the data , Make Word Embeddings from GloVe

## Mainly what I have to do is make a matrix of size (vocab_size, 100) which I can initialize as my weight matrix. Be careful, for now I am assuming no new words can appear in a text other than the ones I have seen before , so later if this works improve it to take in other words too

In [0]:
word_dimensions = 100
import pandas as pd
df = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [0]:
glove = {key: val.values for key, val in df.T.items()}

In [0]:
import nltk
from collections import Counter
import numpy as np

In [0]:
words = []
tokenizer = nltk.tokenize.WordPunctTokenizer()
for a in article:
  tokenized_text = tokenizer.tokenize(a)
  for tokens in tokenized_text:
    words.append(tokens)
    
for h in heading:
  tokenized_text = tokenizer.tokenize(h)
  for tokens in tokenized_text:
    words.append(tokens)

In [0]:
words

['veterans',
 'saluted',
 'worcester',
 "'",
 's',
 'first',
 'ever',
 'breakfast',
 'club',
 'for',
 'ex',
 '-',
 'soldiers',
 'which',
 'won',
 'over',
 'hearts',
 ',',
 'minds',
 'and',
 'bellies',
 '.',
 'the',
 'worcester',
 'breakfast',
 'club',
 'for',
 'hm',
 'forces',
 'veterans',
 'met',
 'at',
 'the',
 'postal',
 'order',
 'in',
 'foregate',
 'street',
 'at',
 '10am',
 'on',
 'saturday',
 '.',
 'the',
 'club',
 'is',
 'designed',
 'to',
 'allow',
 'veterans',
 'a',
 'place',
 'to',
 'meet',
 ',',
 'socialise',
 ',',
 'eat',
 'and',
 'drink',
 ',',
 'giving',
 'hunger',
 'and',
 'loneliness',
 'their',
 'marching',
 'orders',
 '.',
 'father',
 '-',
 'of',
 '-',
 'two',
 'dave',
 'carney',
 ',',
 'aged',
 '43',
 ',',
 'of',
 'merrimans',
 'hill',
 ',',
 'worcester',
 ',',
 'set',
 'up',
 'the',
 'club',
 'after',
 'being',
 'inspired',
 'by',
 'other',
 'similar',
 'clubs',
 'across',
 'the',
 'country',
 '.',
 'he',
 'said',
 ':',
 '"',
 'as',
 'you',
 'can',
 'see',
 'from',

In [0]:
wordcounts = Counter(words).most_common(170000)
vocab_list = np.array([word for word,_ in wordcounts])
dictionary_word_code = {word:code for code,word in enumerate(vocab_list)}

In [0]:
embedding_list = []
i = 0
while i != vocab_list.shape[0] :
  try:
    embedding_list.append(glove[vocab_list[i]])
    i = i+1
  except KeyError as e:
    embedding_list.append(np.random.normal(0,0.1,[word_dimensions]))
    i = i+1
    
  if i%1000 == 0:
    print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [0]:
embedding_matrix = np.vstack(embedding_list)

In [0]:
embedding_matrix

array([[-0.038194  , -0.24487   ,  0.72812   , ..., -0.1459    ,
         0.8278    ,  0.27062   ],
       [-0.33979   ,  0.20941   ,  0.46348   , ..., -0.23394   ,
         0.47298   , -0.028803  ],
       [-0.10767   ,  0.11053   ,  0.59812   , ..., -0.83155   ,
         0.45293   ,  0.082577  ],
       ...,
       [-0.06889619, -0.0915261 ,  0.03377779, ..., -0.05064638,
        -0.03349665, -0.06075078],
       [-0.29514   , -0.88073   , -0.35538   , ..., -0.036788  ,
        -0.23563   ,  0.054821  ],
       [ 0.017643  , -0.090398  , -0.81834   , ..., -0.19481   ,
        -0.87331   ,  0.18943   ]])

**Embedding Matrix is created and in the variable *embedding_matrix***

# Making a Bi-Directional RNN with Attention , no dropout layer

## For now making a 3 Layer LSTM deep network (could cause vanishing gradient problem) and not using the EOS thing, will make the summary a fixed number of words , mostly wont be coherent but lets see if any output comes at all first.

In [0]:
import tensorflow as tf

#### 'The' also has a code of 0 and you are gonna zero pad things so could lead to problems

#### Also you are using too much RAM afterwards change variable names so you overwriting them and not keep making new ones
Ok so for saving RAM I am thinking I will make a pickle file for all the important stuff and load it in so all the useless variables dont take up space

In [0]:
tokenized_articles = []
tokenized_headings = []
for a in article:
  tokens = tokenizer.tokenize(a)
  tokenized_articles.append(tokens)
for h in heading:
  tokens = tokenizer.tokenize(h)
  tokenized_headings.append(tokens)
  
max_length_article = max([len(a) for a in tokenized_articles])
max_length_heading = max([len(h) for h in tokenized_headings])

numerical_articles = []
numerical_headings = []
for a in tokenized_articles:
    list1 = [dictionary_word_code[b] for b in a]
    numerical_articles.append(list1)
for h in tokenized_headings:
  list1 = [dictionary_word_code[b] for b in h]
  numerical_headings.append(list1)
  
padded_numerical_articles = []
padded_numerical_headings = []
for a in numerical_articles:
  zeros = [0]*(max_length_article - len(a))
  list1 = a + zeros
  padded_numerical_articles.append(list1)
for h in numerical_headings:
  zeros = [0]*(max_length_heading - len(h))
  list1 = h + zeros
  padded_numerical_headings.append(list1)
  
with open("input_data.pickle", "wb") as f:
    pickle.dump((embedding_matrix,padded_numerical_articles,padded_numerical_headings,vocab_list), f)

# Too much RAM was used so start from here

In [0]:
import pickle 
with open("input_data.pickle","rb") as f:
  _ , articles , headings, vocab_list = pickle.load(f)

In [0]:
import numpy as np
def next_batch(articles,headings,batch_size,num_time_steps = 100,num_output_steps = 10):                 # 850     #84
  
  snipped_articles = []
  snipped_headings = []
  
  for a in articles:
    part_article = a[0:num_time_steps]
    snipped_articles.append(part_article)
    
  random = np.random.randint(0,len(articles) - batch_size -1)
  batch_x = np.array(snipped_articles[random:random+batch_size]).reshape([batch_size,num_time_steps,1])
  onehotlistarticles = []
  for article in batch_x:
    for num in article:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistarticles.append(zeros)
  onehotlistarticles = np.array(onehotlistarticles).reshape([batch_size,num_time_steps,-1])
  
  for h in headings:
    part_heading = h[0:num_output_steps]
    snipped_headings.append(part_heading)
  
  batch_y = np.array(snipped_headings[random:random+batch_size]).reshape([batch_size,num_output_steps,1])
  onehotlistheadings = []
  for heading in batch_y:
    for num in heading:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistheadings.append(zeros)
  onehotlistheadings = np.array(onehotlistheadings).reshape([batch_size,num_output_steps,-1])
  
  return onehotlistarticles , onehotlistheadings

In [0]:
onehotencodedarticle , onehotencodedheading = next_batch(articles,headings,1,50,10)

In [0]:
onehotencodedarticle

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])

In [0]:
onehotencodedheading

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])

# TensorFlow Graph 

## Encoder 

In [0]:
def length(sequence):
  used = tf.reduce_max(sequence, 2)
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

**See Imext Notebook for explaination of tf.reduce_max and how axis collapses and stuff**

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
num_time_steps = 100
num_output_steps = 10
dimensions_rnn_cell = 100
batch_size = 2

In [0]:
tf.reset_default_graph()

In [0]:
X = tf.placeholder(tf.float32,shape = [None,num_time_steps,len(vocab_list)])

** Right now too tired, not initializing with Embedding Matrix as its a pain , have no clue what the fuck the function wants from me, come back later here if the performance is not so good , see the documentation for initializer in LSTMCell**

In [11]:
lstm_fw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]
lstm_bw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]

outputs, output_state_fw, output_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype='float32', sequence_length = length(X))

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.


** Cool so session crashes after using all the RAM**

** For testing if this is working or not **

In [0]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  batch_x, v = next_batch(articles,headings,batch_size,100,10)
  combined_bi_rnn_output = sess.run(outputs,feed_dict = {X:batch_x})

In [0]:
combined_bi_rnn_output

array([[[-3.2783952e-04, -6.8792311e-04, -9.4794697e-04, ...,
          7.4256625e-04,  9.5902121e-04, -2.3523576e-03],
        [ 7.2198868e-04, -1.6717522e-03, -9.4186328e-04, ...,
         -9.8459411e-04,  2.8810717e-04, -1.9490661e-03],
        [-2.4890318e-04, -2.2269606e-03, -8.7335058e-05, ...,
         -2.5667732e-03, -2.6391957e-05, -8.0228376e-04],
        ...,
        [ 1.0020640e-03,  1.0294392e-03, -8.2223839e-04, ...,
         -2.1115739e-03,  1.8902525e-03,  1.9151869e-04],
        [ 4.7159812e-04,  6.7479024e-04, -8.5636129e-04, ...,
         -1.6434949e-03,  7.3492981e-04,  1.9513143e-03],
        [-4.7449372e-04,  9.3540055e-04,  3.5114659e-04, ...,
         -1.4197577e-03, -2.0201574e-04,  1.1504232e-03]],

       [[ 1.2039823e-03, -4.0472256e-05, -1.8650545e-04, ...,
          2.3192090e-03,  8.3211134e-04,  9.6437446e-04],
        [ 9.0374786e-04, -1.1380591e-03, -8.6801325e-04, ...,
          1.5123325e-03,  1.6813138e-03,  4.9060979e-04],
        [ 1.8986216e-03, 

In [0]:
## Attention Mechanism : For now, wont be taking input from the final output of the network and lets see how it performs, afterwards take as input from the output of the final stage too 

## I think thats possible because the final outputs are also placeholders right? So that can be input to your 'neural_network' function

### Define a Neural Network basically 200 : 100 : 84

In [0]:
combined_BiRNN_outputs = tf.placeholder(tf.float32,shape = [None,num_time_steps,2*dimensions_rnn_cell])

In [0]:
num_hidden_neurons = 100
def neural_network(combined_BiRnn_output):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,2*dimensions_rnn_cell,num_hidden_neurons],0.3,1/200))
    Wx2 = tf.Variable(tf.random_normal([batch_size,num_hidden_neurons,num_output_steps],0.5,1/50))
    b1 = tf.Variable(tf.constant(0.5,shape = [num_hidden_neurons]))
    b2 = tf.Variable(tf.constant(0.3,shape = [num_output_steps]))
    first_out = tf.nn.tanh(tf.add(tf.matmul(combined_BiRnn_output,Wx1),b1))
    e = tf.nn.relu(tf.add(tf.matmul(first_out,Wx2),b2))
        
    return e

In [0]:
def softmax(e_values):
    j = tf.exp(e_values)
    totals = tf.reduce_sum(j,1)
    j = tf.unstack(j,axis = 0)
    totals = tf.unstack(totals, axis = 0)
    for i in range(batch_size):
        j[i] = j[i]/totals[i]
    j = tf.stack(j,axis = 0)
    return j

In [0]:
e_vals = neural_network(combined_BiRNN_outputs)
alpha_values = softmax(e_vals)

In [0]:
init = tf.global_variables_initializer()

** To test whether alpha values are coming or not **

In [0]:
with tf.Session() as sess:
    sess.run(init)
    batch_x, _ = next_batch(articles,headings,batch_size,100,10)
    outs = sess.run(outputs,feed_dict = {X: batch_x})
    
    alpha = sess.run(alpha_values,feed_dict = {combined_BiRNN_outputs:outs})

In [0]:
alpha

array([[[0.01067784, 0.01068803, 0.01068037, ..., 0.01068256,
         0.01068129, 0.01068229],
        [0.01016384, 0.01016348, 0.01016377, ..., 0.01016385,
         0.01016377, 0.01016391],
        [0.0093227 , 0.00930638, 0.00931866, ..., 0.00931506,
         0.00931708, 0.0093158 ],
        ...,
        [0.01075403, 0.01076575, 0.01075705, ..., 0.01075957,
         0.0107582 , 0.01075913],
        [0.01092826, 0.01094368, 0.01093221, ..., 0.01093544,
         0.01093367, 0.01093498],
        [0.01121162, 0.0112332 , 0.01121708, ..., 0.01122183,
         0.01121918, 0.01122093]],

       [[0.00924081, 0.00923804, 0.00923832, ..., 0.00923288,
         0.00924248, 0.00923921],
        [0.00843764, 0.0084326 , 0.00843306, ..., 0.00842303,
         0.00844053, 0.00843464],
        [0.0093433 , 0.00934097, 0.00934119, ..., 0.00933637,
         0.00934468, 0.00934188],
        ...,
        [0.00954852, 0.00954695, 0.00954719, ..., 0.00954347,
         0.00954944, 0.00954765],
        [0.0

** Weighted Average **

In [0]:
def weighted_average(outputs,alphas):
    
    alphas = tf.transpose(alphas,perm = [0,2,1])
    context_vectors = tf.matmul(alphas,outputs)
    
    return context_vectors

In [0]:
context_vectors = weighted_average(combined_BiRNN_outputs,alpha_values)

### Another idea you could do is instead of one hot encoding the input words use the embedding matrix you created, though wont be as useful if you could load up the embedding matrix as the weight vector it may be better than normal one hot encoding

### Or use a LSTM cell as the first layer and top it off with 2 BiRnn layers or something if LSTM cell supports initialzing 


In [0]:
context = tf.placeholder(tf.float32,shape = [batch_size,num_output_steps,2*dimensions_rnn_cell])

In [0]:
cell_factory = tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell,activation=tf.nn.relu)
initial_state = cell_factory.zero_state(batch_size, dtype=tf.float32)
decoder, _ = tf.nn.dynamic_rnn(cell_factory,context,initial_state=initial_state)

In [0]:
def output_neural_network(decoder_rnn):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,dimensions_rnn_cell,len(vocab_list)],0.5,1/dimensions_rnn_cell))
    b1 = tf.Variable(tf.constant(0.5,shape = [len(vocab_list)]))
    first_out = tf.nn.softmax(tf.nn.relu((tf.add(tf.matmul(decoder_rnn,Wx1),b1))))
        
    return first_out

In [0]:
probability_distribution = output_neural_network(decoder)

In [0]:
y_true = tf.placeholder(tf.float32,shape = [None,num_output_steps,len(vocab_list)])

In [0]:
c_e = tf.nn.softmax_cross_entropy_with_logits_v2(logits = probability_distribution,labels = y_true)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.3)
train = optimizer.minimize(c_e)

In [0]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  
  for i in range(100):
    batch_x , batch_y = next_batch(articles,headings,batch_size,100,10)
    outs = sess.run(outputs, feed_dict = {X:batch_x})
    con = sess.run(context_vectors, feed_dict = {combined_BiRNN_outputs:outs})
    sess.run(train, feed_dict = {context:con,y_true:batch_y})
    
    if i%10 ==0:
      print(sess.run(c_e,feed_dict = {context:con,y_true:batch_y}))
      
    batch_x , batch_y = next_batch(articles,headings,1,100,10)
    outs = sess.run(outputs, feed_dict = {X:batch_x})
    con = sess.run(context_vectors, feed_dict = {combined_BiRNN_outputs:outs})
    final_output = sess.run(probability_distribution, feed_dict = {context:con})
      

In [27]:
final_output

array([[[6.0349184e-06, 6.0349184e-06, 6.0350158e-06, ...,
         6.0350076e-06, 6.0349198e-06, 6.0349603e-06],
        [6.0349075e-06, 6.0349130e-06, 6.0350790e-06, ...,
         6.0350717e-06, 6.0349107e-06, 6.0349794e-06],
        [6.0348721e-06, 6.0348857e-06, 6.0350994e-06, ...,
         6.0350985e-06, 6.0348771e-06, 6.0349662e-06],
        ...,
        [6.0347879e-06, 6.0348430e-06, 6.0351495e-06, ...,
         6.0351886e-06, 6.0347998e-06, 6.0349339e-06],
        [6.0347816e-06, 6.0348425e-06, 6.0351535e-06, ...,
         6.0351995e-06, 6.0347943e-06, 6.0349316e-06],
        [6.0347766e-06, 6.0348430e-06, 6.0351558e-06, ...,
         6.0352072e-06, 6.0347902e-06, 6.0349298e-06]],

       [[6.0349948e-06, 6.0350026e-06, 6.0350167e-06, ...,
         6.0349407e-06, 6.0349325e-06, 6.0349430e-06],
        [6.0350299e-06, 6.0350458e-06, 6.0350712e-06, ...,
         6.0349430e-06, 6.0349248e-06, 6.0349389e-06],
        [6.0350235e-06, 6.0350508e-06, 6.0350826e-06, ...,
         6.034